import modules

In [2]:
import pandas as pd
import numpy as np
import math
import glob
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1


목표 : 각 옵션종목을 각 dataframe으로 저장
dataframe이름 예로 201TB305는 콜옵션, 23년, 11월, 행사가 305이므로 -> TB305C
201TC2337는 콜옵션, 23년, 12월, 행사가 337.5이므로 -> TC337C
301TB312는 풋옵션, 23년, 11월, 행사가 312.5이므로 -> TB312P
301TC335는 풋옵션, 23년, 12월, 행사가 335이므로 -> TC335P

load files

In [3]:
folder_path = './option_price/' # 폴더 경로

example combined_data

In [4]:
combined_data = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if combined_data.empty:
                combined_data = selected_rows
            else:
                combined_data = pd.merge(combined_data, selected_rows, on='date', how='outer')

# print(combined_data)


In [5]:
combined_data.head()

,date,CTB305,CTB307,CTB310,CTB312,CTB315,CTB317,CTB320,CTB322,CTB325,...,CTB330,CTB332,CTB335,CTB337,CTB340,CTB342,CTB345,CTB347,CTB350,CTB352
0,2023-10-27 09:30:00,-6.45,-4.97,-3.7,2.65,-1.80,1.18,0.73,-0.43,0.26,...,0.10,0.07,0.05,0.04,0.03,0.02,NaN,0.01,0.01,NaN
1,2023-10-26 09:30:00,8.96,7.16,5.6,4.25,3.11,2.17,1.45,0.92,0.56,...,0.21,0.13,0.08,0.06,0.05,0.03,0.03,0.02,0.02,0.01
2,2023-10-25 09:30:00,14.40,11.90,10.0,8.00,6.38,4.91,3.65,2.59,1.76,...,0.71,0.42,0.25,0.15,0.09,0.06,0.04,0.02,NaN,0.01
3,2023-10-24 09:30:00,-12.60,-10.60,-8.6,-7.00,-5.52,-4.20,-3.09,-2.18,-1.46,...,-0.58,-0.34,-0.20,-0.12,-0.08,-0.05,-0.04,-0.02,0.01,0.01
4,2023-10-23 09:30:00,14.20,12.30,10.2,8.40,6.81,5.28,4.06,2.95,2.08,...,0.91,0.57,0.34,0.21,0.12,0.08,0.05,0.03,0.02,NaN


CTB930 콜옵션 23년 11월물 9시30분 현재가

In [6]:
CTB930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TB')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB930.empty:
                CTB930 = data
            else:
                CTB930 = pd.merge(CTB930, data, on='date', how='outer')
                        
            CTB930 = CTB930[CTB930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 15시15분 데이터만 추출            


In [7]:
CTB930.head()

,date,305,307,310,312,315,317,320,322,325,...,330,332,335,337,340,342,345,347,350,352
0,2023-10-27 09:30:00,-6.45,-4.97,-3.7,2.65,-1.80,1.18,0.73,-0.43,0.26,...,0.10,0.07,0.05,0.04,0.03,0.02,NaN,0.01,0.01,NaN
1,2023-10-26 09:30:00,8.96,7.16,5.6,4.25,3.11,2.17,1.45,0.92,0.56,...,0.21,0.13,0.08,0.06,0.05,0.03,0.03,0.02,0.02,0.01
2,2023-10-25 09:30:00,14.40,11.90,10.0,8.00,6.38,4.91,3.65,2.59,1.76,...,0.71,0.42,0.25,0.15,0.09,0.06,0.04,0.02,NaN,0.01
3,2023-10-24 09:30:00,-12.60,-10.60,-8.6,-7.00,-5.52,-4.20,-3.09,-2.18,-1.46,...,-0.58,-0.34,-0.20,-0.12,-0.08,-0.05,-0.04,-0.02,0.01,0.01
4,2023-10-23 09:30:00,14.20,12.30,10.2,8.40,6.81,5.28,4.06,2.95,2.08,...,0.91,0.57,0.34,0.21,0.12,0.08,0.05,0.03,0.02,NaN


CTB1515 콜 23년 11월 15시15분 현재가

In [8]:
CTB1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S") # 타입 변경
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, # 행사가를 컬럼 이름으로 넣는다.          
             '체결시간': 'date'}) # 컬럼 이름을 date로 바꾼다.
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTB1515.empty:
                CTB1515 = data
            else:
                CTB1515 = pd.merge(CTB1515, data, on='date', how='outer') # date기준으로 병합
            
            CTB1515 = CTB1515[CTB1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출            


In [9]:
CTB1515.head()

,date,CTB305,CTB307,CTB310,CTB312,CTB315,CTB317,CTB320,CTB322,CTB325,...,CTB330,CTB332,CTB335,CTB337,CTB340,CTB342,CTB345,CTB347,CTB350,CTB352
0,2023-10-27 15:15:00,6.65,5.15,3.82,2.72,1.86,1.20,0.73,-0.43,-0.25,...,0.09,-0.05,-0.03,-0.02,0.02,0.02,0.01,NaN,NaN,NaN
1,2023-10-26 15:15:00,-7.15,5.64,4.29,3.15,2.23,1.52,0.99,0.63,0.39,...,0.16,0.11,0.08,0.05,0.03,0.03,0.02,0.01,0.01,0.01
2,2023-10-25 15:15:00,13.15,-11.00,-9.08,-7.30,-5.68,-4.29,3.12,-2.16,1.43,...,0.53,0.31,0.18,0.11,0.06,0.05,0.03,0.02,NaN,0.01
3,2023-10-23 15:15:00,13.00,-10.85,-9.00,-7.39,-5.77,-4.44,-3.30,-2.36,1.62,...,-0.67,-0.40,-0.23,0.15,-0.09,0.06,0.04,0.02,0.02,0.01
4,2023-10-24 15:15:00,NaN,12.60,-10.60,-8.66,7.11,5.58,4.25,3.10,2.17,...,-0.93,-0.56,-0.33,-0.19,-0.12,0.07,0.05,0.03,0.02,0.01


CTC930 콜 23년 12월물 9시30분 현재가

In [10]:
CTC930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            data.bfill(inplace=True)            
            selected_rows = data[data['date'].dt.time == pd.to_datetime('09:30:00').time()]
            # print(selected_rows)
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTC930.empty:
                CTC930 = data
            else:
                CTC930 = pd.merge(CTC930, data, on='date', how='outer') # date기준으로 병합
        
            CTC930 = CTC930[CTC930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 09시30분 데이터만 추출  

In [11]:
CTC930.head()

,date,CTC305,CTC307,CTC310,CTC312,CTC315,CTC317,CTC320,CTC322,CTC325,CTC327,CTC330,CTC332,CTC335,CTC337
0,2023-10-27 09:30:00,-10.55,-9.07,-7.75,-6.47,5.65,4.60,-3.63,-2.86,2.31,-1.79,1.38,1.05,0.78,0.58
1,2023-10-26 09:30:00,13.00,11.05,9.80,8.32,NaN,5.96,4.88,4.03,3.25,2.55,2.00,1.54,1.18,0.88
2,2023-09-26 09:30:00,26.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.30,NaN,8.43,NaN,NaN,5.15
3,2023-10-23 09:30:00,NaN,NaN,13.50,NaN,10.60,9.02,7.85,6.49,5.61,4.50,3.84,3.09,2.33,1.84
4,2023-10-20 09:30:00,NaN,NaN,-13.95,NaN,-11.00,NaN,-7.98,-6.82,-5.67,-4.74,-3.88,-3.13,-2.50,-2.00


CTC1515 콜옵션 23년 12월물 15시15분 현재가

In [12]:
CTC1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/201TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if CTC1515.empty:
                CTC1515 = data
            else:
                CTC1515 = pd.merge(CTC1515, data, on='date', how='outer') # date기준으로 병합
            
            CTC1515 = CTC1515[CTC1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출  

In [13]:
CTC1515.head()

,date,CTC305,CTC307,CTC310,CTC312,CTC315,CTC317,CTC320,CTC322,CTC325,CTC327,CTC330,CTC332,CTC335,CTC337
0,2023-10-26 15:15:00,11.25,9.80,8.49,7.16,6.00,5.00,4.09,3.30,2.62,2.08,1.61,1.23,0.95,0.71
1,2023-10-20 15:15:00,17.90,16.30,14.30,NaN,11.05,9.67,-8.25,7.06,-5.94,4.94,4.00,3.26,-2.57,-2.03
2,2023-09-07 15:15:00,33.80,NaN,29.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.65,NaN
3,2023-10-27 15:15:00,NaN,9.43,7.95,NaN,5.61,-4.54,-3.68,2.95,2.33,1.83,1.38,1.05,-0.77,0.57
4,2023-10-23 15:15:00,NaN,NaN,12.90,NaN,9.79,8.45,NaN,5.98,4.95,4.01,3.27,-2.54,2.06,1.57


PTB930 풋옵션 23년 11월물 9시30분 현재가

In [14]:
PTB930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTB930.empty:
                PTB930 = data
            else:
                PTB930 = pd.merge(PTB930, data, on='date', how='outer') # date기준으로 병합
            
            PTB930 = PTB930[PTB930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 15시15분 데이터만 추출  


In [15]:
PTB930.head()

,date,PTB305,PTB307,PTB310,PTB312,PTB315,PTB317,PTB320,PTB322,PTB325,...,PTB330,PTB332,PTB335,PTB337,PTB340,PTB342,PTB345,PTB347,PTB350,PTB352
0,2023-10-27 09:30:00,3.42,4.42,5.64,7.06,8.70,10.65,12.50,15.05,17.60,...,22.25,24.45,-25.75,NaN,NaN,33.4,NaN,NaN,NaN,NaN
1,2023-10-26 09:30:00,-2.43,-3.18,-4.11,-5.23,-6.49,-8.02,-9.85,-11.80,-14.20,...,-18.55,-21.45,NaN,-25.95,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25 09:30:00,-1.05,-1.44,-1.95,-2.61,-3.44,-4.45,-5.69,-7.03,-8.88,...,-12.00,-14.90,-17.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-24 09:30:00,1.63,2.14,2.78,3.57,4.55,5.72,7.14,8.80,10.50,...,14.60,16.15,19.25,21.45,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-23 09:30:00,-1.61,-2.07,-2.65,-3.35,-4.22,-5.25,-6.43,-7.85,-9.56,...,-12.85,-14.95,-17.65,-20.00,-22.2,NaN,NaN,NaN,NaN,NaN


PTB1515 풋옵션 23년 11월물 15시15분 현재가

In [16]:
PTB1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TB305_352.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})            
            # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTB1515.empty:
                PTB1515 = data
            else:
                PTB1515 = pd.merge(PTB1515, data, on='date', how='outer') # date기준으로 병합
            PTB1515 = PTB1515[PTB1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출            


In [17]:
PTB1515.head()

,date,PTB305,PTB307,PTB310,PTB312,PTB315,PTB317,PTB320,PTB322,PTB325,...,PTB330,PTB332,PTB335,PTB337,PTB340,PTB342,PTB345,PTB347,PTB350,PTB352
0,2023-10-27 15:15:00,-3.04,-3.99,-5.14,-6.56,-8.23,10.05,-12.10,14.30,-16.60,...,NaN,NaN,26.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-26 15:15:00,-3.56,-4.52,-5.69,-7.04,-8.63,-10.35,-12.25,-14.35,-16.75,...,21.55,23.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25 15:15:00,-1.13,1.55,-2.10,-2.81,3.70,4.79,6.11,7.63,9.48,...,13.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-24 15:15:00,1.08,-1.45,-1.93,2.55,3.33,4.28,5.42,6.84,8.52,...,12.25,14.30,16.70,NaN,21.3,NaN,26.45,NaN,NaN,NaN
4,2023-10-23 15:15:00,1.78,2.30,2.95,3.74,4.70,5.85,7.24,-8.78,10.50,...,14.30,16.70,18.95,21.45,23.8,NaN,NaN,NaN,33.55,NaN


PTC930 풋 23년 12월물 9시30분 현재가

In [18]:
PTC930 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = "C" + csv_file_name.split('.')[0].split('201')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = "P" + csv_file_name.split('.')[0].split('301')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
             # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTC930.empty:
                PTC930 = data
            else:
                PTC930 = pd.merge(PTC930, data, on='date', how='outer') # date기준으로 병합
            PTC930 = PTC930[PTC930['date'].dt.time == pd.to_datetime('09:30:00').time()] # 09시30분 데이터만 추출    

In [19]:
PTC930.head()

,date,PTC305,PTC307,PTC310,PTC312,PTC315,PTC317,PTC320,PTC322,PTC325,PTC327,PTC330,PTC332,PTC335,PTC337
0,2023-10-27 09:30:00,6.62,7.66,8.77,9.99,11.45,-12.2,14.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-26 09:30:00,-5.61,-6.41,-7.39,-8.66,-10.00,NaN,-12.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25 09:30:00,-3.54,-4.12,-4.80,-5.52,-6.50,-7.3,-8.45,-9.8,NaN,NaN,-13.85,NaN,NaN,NaN
3,2023-10-24 09:30:00,4.15,4.79,5.72,NaN,NaN,8.7,9.94,NaN,12.6,NaN,NaN,NaN,20.2,NaN
4,2023-10-23 09:30:00,-3.94,-4.62,-5.30,-6.13,-7.06,-8.2,NaN,NaN,NaN,-13.3,NaN,NaN,-18.4,NaN


PTC1515 풋 23년 12월물 15시15분 현재가

In [20]:
PTC1515 = pd.DataFrame() # 합칠 CSV 파일들을 저장할 빈 DataFrame 생성
root_directory = './option_price/301TC305_337.5/' # 시작 디렉토리 설정

# root_directory와 그 하위 모든 디렉토리에서 CSV 파일을 찾아 합침
for root, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            # CSV 파일을 DataFrame으로 읽어들임
            data = pd.read_csv(file_path)
            csv_file_name = os.path.basename(file_path)
            if csv_file_name.startswith('201'): # 201로 시작하면 끝에 C를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            elif csv_file_name.startswith('301'): # 301로 시작하면 끝에 P를 붙인다.
                csv_file_name = csv_file_name.split('.')[0].split('TC')[1]
            data['체결시간'] = pd.to_datetime(data['체결시간'], format="%Y%m%d%H%M%S")
            data = data[['체결시간', '현재가']].rename(columns={'현재가': csv_file_name, '체결시간': 'date'})
            
             # 읽어들인 데이터를 기존 데이터프레임과 결합
            if PTC1515.empty:
                PTC1515 = data
            else:
                PTC1515 = pd.merge(PTC1515, data, on='date', how='outer') # date기준으로 병합
            PTC1515 = PTC1515[PTC1515['date'].dt.time == pd.to_datetime('15:15:00').time()] # 15시15분 데이터만 추출    


In [21]:
PTC1515.head()

,date,305,307,310,312,315,317,320,322,325,327,330,332,335,337
0,2023-10-27 15:15:00,-6.38,-7.30,-8.49,NaN,NaN,NaN,14.10,15.90,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-26 15:15:00,-6.77,7.74,8.92,10.05,11.55,NaN,14.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25 15:15:00,3.75,NaN,5.11,NaN,6.87,7.86,9.07,NaN,11.85,NaN,15.25,NaN,NaN,NaN
3,2023-10-24 15:15:00,3.56,4.15,4.89,5.57,6.45,7.42,8.55,9.76,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-23 15:15:00,-4.33,5.04,5.78,NaN,NaN,8.70,NaN,NaN,NaN,NaN,16.25,NaN,NaN,NaN


strike price set

In [22]:
strike_price = pd.read_csv('./old_dataset/strike_price.csv')

In [23]:
strike_price

,일자,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
0,20231020,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
1,20231019,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
2,20231018,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
3,20231017,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
4,20231016,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
5,20231013,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
6,20231012,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
7,20231011,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
8,20231010,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
9,20231006,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5


In [24]:
c9sp = strike_price
c15sp = strike_price
p9sp = strike_price
p15sp = strike_price

In [25]:
a = 322

CTB930.loc[:,str(a)]

0      -0.43
1       0.92
2       2.59
3      -2.18
4       2.95
5      -3.33
6       7.93
7       6.14
8      -8.57
9       6.47
10     -9.33
11      7.95
12      7.40
13     -4.80
14      4.85
15     -7.17
16      5.40
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23     22.00
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
30       NaN
31       NaN
32       NaN
33       NaN
34       NaN
35       NaN
36       NaN
37       NaN
38       NaN
39       NaN
40       NaN
41       NaN
42       NaN
43       NaN
44       NaN
45       NaN
46       NaN
47       NaN
48       NaN
49       NaN
805      NaN
811      NaN
Name: 322, dtype: float64

In [26]:
for index, row in strike_price.iterrows():
    for column in strike_price.columns:
        if column != '일자':
            value = row[column]
            value = math.floor(value)
            print(row)
            # print(value)
            # print(CTB930.loc[:,str(value)])
            # print(CTB930.loc[:,CTB930.columns == value])

일자       20231020.0
기준행사가         322.5
S-4           312.5
S-3           315.0
S-2           317.5
S-1           320.0
S+1           325.0
S+2           327.5
S+3           330.0
S+4           332.5
Name: 0, dtype: float64
일자       20231020.0
기준행사가         322.5
S-4           312.5
S-3           315.0
S-2           317.5
S-1           320.0
S+1           325.0
S+2           327.5
S+3           330.0
S+4           332.5
Name: 0, dtype: float64
일자       20231020.0
기준행사가         322.5
S-4           312.5
S-3           315.0
S-2           317.5
S-1           320.0
S+1           325.0
S+2           327.5
S+3           330.0
S+4           332.5
Name: 0, dtype: float64
일자       20231020.0
기준행사가         322.5
S-4           312.5
S-3           315.0
S-2           317.5
S-1           320.0
S+1           325.0
S+2           327.5
S+3           330.0
S+4           332.5
Name: 0, dtype: float64
일자       20231020.0
기준행사가         322.5
S-4           312.5
S-3           315.0
S-2           317.5
S-1 

일자       20230829.0
기준행사가         332.5
S-4           322.5
S-3           325.0
S-2           327.5
S-1           330.0
S+1           335.0
S+2           337.5
S+3           340.0
S+4           342.5
Name: 33, dtype: float64
일자       20230829.0
기준행사가         332.5
S-4           322.5
S-3           325.0
S-2           327.5
S-1           330.0
S+1           335.0
S+2           337.5
S+3           340.0
S+4           342.5
Name: 33, dtype: float64
일자       20230829.0
기준행사가         332.5
S-4           322.5
S-3           325.0
S-2           327.5
S-1           330.0
S+1           335.0
S+2           337.5
S+3           340.0
S+4           342.5
Name: 33, dtype: float64
일자       20230829.0
기준행사가         332.5
S-4           322.5
S-3           325.0
S-2           327.5
S-1           330.0
S+1           335.0
S+2           337.5
S+3           340.0
S+4           342.5
Name: 33, dtype: float64
일자       20230829.0
기준행사가         332.5
S-4           322.5
S-3           325.0
S-2           327.5


In [27]:
CTB930.head()

,date,305,307,310,312,315,317,320,322,325,...,330,332,335,337,340,342,345,347,350,352
0,2023-10-27 09:30:00,-6.45,-4.97,-3.7,2.65,-1.80,1.18,0.73,-0.43,0.26,...,0.10,0.07,0.05,0.04,0.03,0.02,NaN,0.01,0.01,NaN
1,2023-10-26 09:30:00,8.96,7.16,5.6,4.25,3.11,2.17,1.45,0.92,0.56,...,0.21,0.13,0.08,0.06,0.05,0.03,0.03,0.02,0.02,0.01
2,2023-10-25 09:30:00,14.40,11.90,10.0,8.00,6.38,4.91,3.65,2.59,1.76,...,0.71,0.42,0.25,0.15,0.09,0.06,0.04,0.02,NaN,0.01
3,2023-10-24 09:30:00,-12.60,-10.60,-8.6,-7.00,-5.52,-4.20,-3.09,-2.18,-1.46,...,-0.58,-0.34,-0.20,-0.12,-0.08,-0.05,-0.04,-0.02,0.01,0.01
4,2023-10-23 09:30:00,14.20,12.30,10.2,8.40,6.81,5.28,4.06,2.95,2.08,...,0.91,0.57,0.34,0.21,0.12,0.08,0.05,0.03,0.02,NaN


In [28]:
c9sp.describe()

,일자,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
count,5.400000e+01,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,2.023089e+07,332.685185,322.685185,325.185185,327.685185,330.185185,335.185185,337.685185,340.185185,342.685185
std,8.013054e+01,7.583682,7.583682,7.583682,7.583682,7.583682,7.583682,7.583682,7.583682,7.583682
min,2.023073e+07,317.500000,307.500000,310.000000,312.500000,315.000000,320.000000,322.500000,325.000000,327.500000
25%,2.023082e+07,327.500000,317.500000,320.000000,322.500000,325.000000,330.000000,332.500000,335.000000,337.500000
50%,2.023091e+07,332.500000,322.500000,325.000000,327.500000,330.000000,335.000000,337.500000,340.000000,342.500000
75%,2.023093e+07,337.500000,327.500000,330.000000,332.500000,335.000000,340.000000,342.500000,345.000000,347.500000
max,2.023102e+07,350.000000,340.000000,342.500000,345.000000,347.500000,352.500000,355.000000,357.500000,360.000000


In [29]:
for index, row in c9sp.iterrows():
    for column in c9sp.columns:
        if column != '일자':
            value = row[column]
            value = math.floor(value)            
            CTB930['']
            print(c9sp.loc[index,column])
            # print(value)

KeyError: ''

In [ ]:
sp12.columns.drop('일자')

Index(['기준행사가', 'S-4', 'S-3', 'S-2', 'S-1', 'S+1', 'S+2', 'S+3', 'S+4'], dtype='object')

In [ ]:
for i, c in sp12.index, sp12.columns.drop('일자'):
    print(sp12.loc[i,c])

ValueError: too many values to unpack (expected 2)

In [ ]:
for in sp
sp12

,일자,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
0,20231020,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
1,20231019,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
2,20231018,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
3,20231017,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
4,20231016,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
5,20231013,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
6,20231012,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
7,20231011,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
8,20231010,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
9,20231006,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
